In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw


_image_ids = []
image_info = []
# Background is always the first class
class_info = [{"source": "", "id": 0, "name": "BG"}]
source_class_ids = {}

subset='train'
data_dir='../../datasets/food/'

In [ ]:
def add_image(source, image_id, path, **kwargs):
    info = {
        "id": image_id,
        "source": source,
        "path": path,
    }
    info.update(kwargs)
    image_info.append(info)

def add_class(source, class_id, class_name):
    assert "." not in source, "Source name cannot contain a dot"
    # Does the class exist already?
    for info in class_info:
        if info['source'] == source and info["id"] == class_id:
            # source.class_id combination already available, skip
            return
    # Add the class
    class_info.append({
        "source": source,
        "id": class_id,
        "name": class_name,
    })
    
def load_mask(image_id):
    """Load instance masks for the given image.

    Different datasets use different ways to store masks. Override this
    method to load instance masks and return them in the form of am
    array of binary masks of shape [height, width, instances].

    Returns:
        masks: A bool array of shape [height, width, instance count] with
            a binary mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
    """
    # Override this function to load a mask from your dataset.
    # Otherwise, it returns an empty mask.
    logging.warning("You are using the default load_mask(), maybe you need to define your own one.")
    mask = np.empty([0, 0, 0])
    class_ids = np.empty([0], np.int32)
    
    print(mask)
    print(class_ids)
    
    return mask, class_ids

### load balloon

In [ ]:
foodlist=["蛋", "豬肉","雞肉","牛肉","魚","蝦子","茄子","豆腐",'鴨肉','地瓜',
          '杏鮑菇','洋芋',"南瓜","紅蘿蔔",'小黃瓜','苦瓜',"青椒","香菇",'白蘿蔔',
          "白飯",'青菜','玉米','豆芽']

for idx,element in enumerate(foodlist,1):
    add_class("food", idx,element)
    
# add_class("balloon", 1, "balloon")
# add_class("pizza", 2, "pizza")
# add_class("undefined", 3, "undefined")
# add_class("food", 1, "蛋")
# add_class("food", 2, "豬肉")
# add_class("food", 3, "雞肉")
# add_class("food", 4, "牛肉")
# add_class("food", 5, "魚")
# add_class("food", 7, "茄子")
# add_class("food", 8, "豆腐")
# add_class("food", 13, "南瓜")
# add_class("food", 14, "紅蘿蔔")
# add_class("food", 18, "香菇")
# add_class("food", 50, "undefined")
class_info

In [ ]:
# 資料集路徑
assert subset in ["train", "val"]
dataset_dir = os.path.join(data_dir, subset)
print(dataset_dir)

In [ ]:
# 載入標註json檔
annotations = json.load(open(os.path.join(dataset_dir, "1-2800.json")))
print('number of images :{}'.format(len(annotations)))
print(type(annotations))

In [ ]:
# 取圖片檔資訊 dict轉成list
annotations = list(annotations.values())  # don't need the dict keys
#print(annotations)

In [ ]:
# 過濾沒有標註區塊的圖
# The VIA tool saves images in the JSON even if they don't have any
# annotations. Skip unannotated images.
annotations = [a for a in annotations if a['regions']]
print('annotated of images :{}'.format(len(annotations)))
#print(annotations)

In [ ]:
a=annotations[46]
# print(a)
# print(a['regions'])

for r in a['regions']:
   # print(r['shape_attributes'])
    print(r['region_attributes'])

In [ ]:
dict_class={}
for i in class_info:
    dict_class[i['name']]=i['id']
dict_class

In [ ]:
# Add images
for a in annotations:
    # Get the x, y coordinaets of points of the polygons that make up
    # the outline of each object instance. There are stores in the
    # shape_attributes (see json format above)
    if type(a['regions']) is dict:
        polygons = [r['shape_attributes'] for r in a['regions'].values()]        
        #print(polygons)
    else:
        polygons = [r['shape_attributes'] for r in a['regions']]         
        #print(polygons)
        
    objects = [s['region_attributes'] for s in a['regions']]  
    class_ids=[]
    for n in objects:
        if 'name' in n.keys():
            class_ids.append(dict_class[n['name']])        
#         else:
#             class_ids.append(dict_class['undefined'])
    # load_mask() needs the image size to convert polygons to masks.
    # Unfortunately, VIA doesn't include it in JSON, so we must read
    # the image. This is only managable since the dataset is tiny.   
    image_path = os.path.join(dataset_dir, a['filename'])
    #print(image_path)
    image = skimage.io.imread(image_path)
    #print(image)
    height, width = image.shape[:2]
    #print(height, width)
    add_image(
        "food",
        image_id=a['filename'],  # use file name as a unique image id
        path=image_path,
        width=width, height=height,
        polygons=polygons,
        class_ids=class_ids)
    # append data information to dict image_info 

In [ ]:
image_info

In [ ]:
# for a in annotations:
#     objects = [s['region_attributes'] for s in a['regions']]
#     #print(objects) 
#     class_ids=[]
#     for n in objects:
#         if n:
#             class_ids.append(dict_class[n['name']])        
#         else:
#             class_ids.append(4)
#     print(class_ids)

### load mask

In [ ]:
image_id=42
# 取某張照片資訊
info = image_info[image_id]
print(info)

In [ ]:
# 判斷資料集來源 
if info["source"] != "food":
    load_mask(image_id)

In [ ]:
# 區塊類別list
class_ids = info['class_ids']
class_ids = np.array(class_ids)
type(class_ids)

In [ ]:
# 用長,寬,實體數 建立mask點陣圖
# Convert polygons to a bitmap mask of shape
# [height, width, instance_count]
mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                 dtype=np.uint8)
print(mask.shape)

In [ ]:
# info['polygons']取多邊形資訊並編號
for i, p in enumerate(info["polygons"]):
    print(i)
    print(p)   

In [ ]:
# 取多邊形資訊並編號 
for i, p in enumerate(info["polygons"]):
    # Get indexes of pixels inside the polygon and set them to 1
    # 根據頂點座標計算區塊內座標點 設值=1
    rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
    print('row',rr)
    print('column',cc)
    mask[rr, cc, i] = 1

In [ ]:
# Return mask, and array of class IDs of each instance. Since we have
# one class ID only, we return an array of 1s
# return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

# mask.astype(np.bool)
# mask.astype(np.bool)
# array of class IDs   mask.shape[-1]:  (512, 383, 1) mask第三維度代表區塊數
# np.ones([mask.shape[-1]], dtype=np.int32)

In [ ]:
 def image_reference(self, image_id):
    """Return the path of the image."""
    info = self.image_info[image_id]
    if info["source"] == "balloon":
        return info["path"]
    else:
        super(self.__class__, self).image_reference(image_id)

### no apply

In [ ]:
# https://cloud.tencent.com/developer/section/1414714
from skimage.draw import polygon
img = np.zeros((10, 10), dtype=np.uint8)
r = np.array([1, 2, 8, 1])
c = np.array([1, 7, 4, 1])
rr, cc = polygon(r, c)
print(rr)
print(cc)
img[rr, cc] = 1
img

In [ ]:
class Base(object):
    def __init__(self):
        print("enter Base")
        print("leave Base")

class A(Base):
    def __init__(self):
        print("enter A")
        super(A, self).__init__()
        print("leave A")

class B(Base):
    def __init__(self):
        print("enter B")
        super(B, self).__init__()
        print("leave B")

class C(A, B):
    def __init__(self):
        print("enter C")
        super(C, self).__init__()
        print("leave C")

In [ ]:
C.mro()

In [ ]:
c=C()

In [ ]:
class FooParent(object):
    def __init__(self):
        self.parent = 'I\'m the parent.'
        print ('Parent')
    
    def bar(self,message):
        print ("%s from Parent" % message)
 
class FooChild(FooParent):
    def __init__(self):
        # super(FooChild,self) 首先找到 FooChild 的父类（就是类 FooParent），
        # 然后把类 FooChild 的对象转换为类 FooParent 的对象
        super(FooChild,self).__init__()    
        print ('Child')
        
    def bar(self,message):
        super(FooChild, self).bar(message)
        print ('Child bar fuction')
        print (self.parent)
 
if __name__ == '__main__':
    fooChild = FooChild()
    fooChild.bar('HelloWorld')

In [ ]:
h={'a':{'k':{0}},'b':{'k':{1}},'c':{'k':{2}},'d':{'k':{3}},'e':{'k':{}}}

h=list(h.values())
#print(h)

for d in h:
#     if d['k']:
        print(d['k'])

    

In [ ]:
s={}

bool(s)

In [ ]:
def load_balloon(dataset_dir, subset):
    """Load a subset of the Balloon dataset.
    dataset_dir: Root directory of the dataset.
    subset: Subset to load: train or val
    """
    # Add classes. We have only one class to add.
    add_class("balloon", 1, "balloon")

    # Train or validation dataset?
    assert subset in ["train", "val"]
    dataset_dir = os.path.join(dataset_dir, subset)

    # Note: In VIA 2.0, regions was changed from a dict to a list.
    annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
    annotations = list(annotations.values())  # don't need the dict keys

    # The VIA tool saves images in the JSON even if they don't have any
    # annotations. Skip unannotated images.
    annotations = [a for a in annotations if a['regions']]

    # Add images
    for a in annotations:
        # Get the x, y coordinaets of points of the polygons that make up
        # the outline of each object instance. These are stores in the
        # shape_attributes (see json format above)
        # The if condition is needed to support VIA versions 1.x and 2.x.
        if type(a['regions']) is dict:
            polygons = [r['shape_attributes'] for r in a['regions'].values()]
        else:
            polygons = [r['shape_attributes'] for r in a['regions']] 

        # load_mask() needs the image size to convert polygons to masks.
        # Unfortunately, VIA doesn't include it in JSON, so we must read
        # the image. This is only managable since the dataset is tiny.
        image_path = os.path.join(dataset_dir, a['filename'])
        image = skimage.io.imread(image_path)
        height, width = image.shape[:2]

        self.add_image(
            "balloon",
            image_id=a['filename'],  # use file name as a unique image id
            path=image_path,
            width=width, height=height,
            polygons=polygons)

In [ ]:
# https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/utils.py
# balloon.py
# class BalloonDataset(utils.Dataset):
def add_image(source, image_id, path, **kwargs):
    image_info = {
        "id": image_id,
        "source": source,
        "path": path,
    }
    image_info.update(kwargs)
    image_info.append(image_info)

def image_reference(image_id):
    """Return a link to the image in its source Website or details about
    the image that help looking it up or debugging it.

    Override for your dataset, but pass to this function
    if you encounter images not in your dataset.
    """
    return ""

def prepare(class_map=None):
    """Prepares the Dataset class for use.

    TODO: class map is not supported yet. When done, it should handle mapping
          classes from different datasets to the same class ID.
    """

    def clean_name(name):
        """Returns a shorter version of object names for cleaner display."""
        return ",".join(name.split(",")[:1])

    # Build (or rebuild) everything else from the info dicts.
    self.num_classes = len(self.class_info)
    self.class_ids = np.arange(self.num_classes)
    self.class_names = [clean_name(c["name"]) for c in self.class_info]
    self.num_images = len(self.image_info)
    self._image_ids = np.arange(self.num_images)

    # Mapping from source class and image IDs to internal IDs
    self.class_from_source_map = {"{}.{}".format(info['source'], info['id']): id
                                  for info, id in zip(self.class_info, self.class_ids)}
    self.image_from_source_map = {"{}.{}".format(info['source'], info['id']): id
                                  for info, id in zip(self.image_info, self.image_ids)}

    # Map sources to class_ids they support
    self.sources = list(set([i['source'] for i in self.class_info]))
    self.source_class_ids = {}
    # Loop over datasets
    for source in self.sources:
        self.source_class_ids[source] = []
        # Find classes that belong to this dataset
        for i, info in enumerate(self.class_info):
            # Include BG class in all datasets
            if i == 0 or source == info['source']:
                self.source_class_ids[source].append(i)

def map_source_class_id(source_class_id):
    """Takes a source class ID and returns the int class ID assigned to it.

    For example:
    dataset.map_source_class_id("coco.12") -> 23
    """
    return self.class_from_source_map[source_class_id]

def get_source_class_id(class_id, source):
    """Map an internal class ID to the corresponding class ID in the source dataset."""
    info = self.class_info[class_id]
    assert info['source'] == source
    return info['id']

@property
def image_ids():
    return self._image_ids

def source_image_link(image_id):
    """Returns the path or URL to the image.
    Override this to return a URL to the image if it's available online for easy
    debugging.
    """
    return self.image_info[image_id]["path"]

def load_image(image_id):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(self.image_info[image_id]['path'])
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image

def load_mask(image_id):
    """Load instance masks for the given image.

    Different datasets use different ways to store masks. Override this
    method to load instance masks and return them in the form of am
    array of binary masks of shape [height, width, instances].

    Returns:
        masks: A bool array of shape [height, width, instance count] with
            a binary mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
    """
    # Override this function to load a mask from your dataset.
    # Otherwise, it returns an empty mask.
    logging.warning("You are using the default load_mask(), maybe you need to define your own one.")
    mask = np.empty([0, 0, 0])
    class_ids = np.empty([0], np.int32)
    return mask, class_ids

In [ ]:
def load_mask(self, image_id):
    """Generate instance masks for an image.
    Returns:
    masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
    class_ids: a 1D array of class IDs of the instance masks.
    """
    # If not a balloon dataset image, delegate to parent class.
    image_info = self.image_info[image_id]
    if image_info["source"] != "balloon":
        return super(self.__class__, self).load_mask(image_id)

    # Convert polygons to a bitmap mask of shape
    # [height, width, instance_count]
    info = self.image_info[image_id]
    mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                    dtype=np.uint8)
    for i, p in enumerate(info["polygons"]):
        # Get indexes of pixels inside the polygon and set them to 1
        rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
        mask[rr, cc, i] = 1

    # Return mask, and array of class IDs of each instance. Since we have
    # one class ID only, we return an array of 1s
    return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

### end